In [1]:
import os
import pandas as pd
from pathlib import Path

# Set project root and define directoriesproject_root = Path(...).resolve()
os.chdir(project_root)

data_dir = project_root / "03_training"
output_dir = project_root / "04_prediction"

# Load the trained future games
trained_future_games_path = data_dir / "trained_future_games.csv"
future_games = pd.read_csv(trained_future_games_path)

In [2]:
future_games

,Wk,Day,Date,Time,Venue,Team,Opponent,Hour,Day_Code,Venue_Code,Opp_Code,GF_Rolling,GA_Rolling,Sh_Rolling,SoT_Rolling,Dist_Rolling,FK_Rolling,PK_Rolling,PKatt_Rolling
0,19.0,Fri,2025-01-10,21:00,Estadio del Rayo Vallecano,Rayo Vallecano,Celta Vigo,21,4,1,6,1.666667,1.666667,14.000000,3.666667,18.200000,0.666667,0.000000,0.000000
1,19.0,Sat,2025-01-11,14:00,Estadio de Mendizorroza,Alaves,Girona,14,5,1,12,1.666667,1.666667,7.333333,2.666667,16.966667,0.666667,0.333333,0.333333
2,19.0,Sat,2025-01-11,16:15,Estadio Municipal José Zorrilla,Valladolid,Real Betis,16,5,1,5,0.666667,1.666667,4.000000,1.333333,28.466667,0.333333,0.000000,0.000000
3,19.0,Sat,2025-01-11,18:30,RCDE Stadium,Espanyol,Leganes,18,5,1,16,0.333333,0.666667,9.333333,2.666667,21.200000,0.000000,0.000000,0.000000
4,19.0,Sat,2025-01-11,21:00,Estadio Ramón Sánchez Pizjuán,Sevilla,Valencia,21,5,1,24,2.000000,2.666667,7.333333,3.333333,19.200000,0.000000,0.000000,0.000000
5,19.0,Sun,2025-01-12,13:00,Estadio de Gran Canaria,Las Palmas,Getafe,13,6,1,11,1.000000,0.333333,10.000000,3.666667,21.833333,1.333333,0.000000,0.000000
6,19.0,Sun,2025-01-12,16:15,Riyadh Air Metropolitan Stadium,Atletico Madrid,Osasuna,16,6,1,19,2.333333,1.333333,10.000000,5.333333,20.466667,0.333333,0.000000,0.000000
7,19.0,Mon,2025-01-13,21:00,Reale Arena,Real Sociedad,Villarreal,21,0,1,26,1.000000,0.666667,12.333333,3.333333,17.766667,0.000000,0.000000,0.000000
8,20.0,Fri,2025-01-17,21:00,RCDE Stadium,Espanyol,Valladolid,21,4,1,25,0.333333,0.666667,9.333333,2.666667,21.200000,0.000000,0.000000,0.000000
9,20.0,Sat,2025-01-18,14:00,Estadio El Sadar,Osasuna,Rayo Vallecano,14,5,1,20,1.000000,1.333333,9.333333,3.666667,20.533333,0.333333,0.000000,0.000000


In [8]:
import joblib

# Load the trained model
model_path = "random_forest_model.pkl"
rf = joblib.load(model_path)
print("Model loaded successfully")

# Load predictors
predictors_path = "predictors.pkl"
predictors = joblib.load(predictors_path)
print(f"Predictors loaded: {predictors}")

Model loaded successfully
Predictors loaded: ['Venue_Code', 'Opp_Code', 'Hour', 'Day_Code', 'GF_Rolling', 'GA_Rolling', 'Sh_Rolling', 'SoT_Rolling', 'Dist_Rolling', 'FK_Rolling', 'PK_Rolling', 'PKatt_Rolling']


In [13]:
# Calculate probabilities for home and away wins
future_games["Home_Win_Probability"] = (rf.predict_proba(future_games[predictors])[:, 1] * 100).round(2)
future_games["Away_Win_Probability"] = (100 - future_games["Home_Win_Probability"]).round(2)

# Determine the predicted winner
future_games["Predicted_Winner"] = future_games.apply(
    lambda row: row["Team"] if row["Home_Win_Probability"] > 50 else row["Opponent"], axis=1
)

# Create results table with relevant columns
results_table = future_games[["Team","Home_Win_Probability", "Opponent", "Away_Win_Probability", "Predicted_Winner"]]
results_table = results_table.rename(columns={
    "Team": "Home", 
    "Home_Win_Probability": "Home Win %", 
    "Opponent": "Away", 
    "Away_Win_Probability": "Away Win %",
    "Predicted_Winner": "Predicted Winner"
})

# Save md19_md20_predictions.csv
predictions = output_dir / "md19_md20_predictions.csv"
results_table.to_csv(predictions, index=False)
print("Saved to md19_md20_predictions.csv")

Saved to md19_md20_predictions.csv


In [14]:
results_table

,Home,Home Win %,Away,Away Win %,Predicted Winner
0,Rayo Vallecano,61.08,Celta Vigo,38.92,Rayo Vallecano
1,Alaves,39.91,Girona,60.09,Girona
2,Valladolid,23.91,Real Betis,76.09,Real Betis
3,Espanyol,40.45,Leganes,59.55,Leganes
4,Sevilla,40.62,Valencia,59.38,Valencia
5,Las Palmas,34.68,Getafe,65.32,Getafe
6,Atletico Madrid,55.96,Osasuna,44.04,Atletico Madrid
7,Real Sociedad,51.91,Villarreal,48.09,Real Sociedad
8,Espanyol,34.90,Valladolid,65.10,Valladolid
9,Osasuna,54.21,Rayo Vallecano,45.79,Osasuna
